# 0) 필수 라이브러리 & 한글 폰트 설정

In [ ]:
!apt-get update -qq
!apt-get install -y fonts-nanum > /dev/null

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# 한글 폰트 설정
plt.rc("font", family="NanumGothic")
plt.rcParams["axes.unicode_minus"] = False

# 1) 데이터 불러오기
##  - 아래 path를 본인 파일명으로 교체하세요.

In [ ]:
FILE_PATH = "/content/행정구역_시군구_별__성별_인구수_20250923133637.xlsx"

df_raw = pd.read_excel(FILE_PATH, engine="openpyxl")

# 2) 컬럼 정규화 & 값 컬럼 자동 탐지
##   - KOSIS/통계청 양식 변동에 대응: 마지막 "숫자형" 컬럼을 값으로 인식

In [ ]:
df = df_raw.copy()
df.columns = [str(c).strip() for c in df.columns]

# 흔히 쓰이는 후보 열 이름(데이터셋에 따라 다를 수 있어 유연 처리)
possible_region_cols = [
    "행정구역(시군구)별", "시군구", "지역", "행정구역", "행정구역별"
]
possible_gender_cols = ["성별", "남여", "성별(계,남,여)"]

def pick_first_existing(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

col_region = pick_first_existing(df.columns, possible_region_cols)
col_gender = pick_first_existing(df.columns, possible_gender_cols)

if col_region is None:
    raise ValueError(f"시군구 컬럼을 찾을 수 없습니다. 후보: {possible_region_cols}\n현재 컬럼: {list(df.columns)}")

# 값(인구수) 컬럼: 숫자 비율이 높은 '마지막' 컬럼을 기본값으로 선택
numeric_ratio = []
for c in df.columns:
    try:
        vals = pd.to_numeric(df[c], errors="coerce")
        ratio = vals.notna().mean()
        numeric_ratio.append((c, ratio))
    except Exception:
        numeric_ratio.append((c, 0.0))

# 숫자 비율 높은 컬럼 중에서 가장 마지막에 있는 컬럼을 선택
candidate_value_cols = [c for c, r in numeric_ratio if r > 0.5]
if not candidate_value_cols:
    raise ValueError("숫자형 값(인구수) 컬럼을 찾지 못했습니다. 엑셀 시트를 확인해 주세요.")
col_value = candidate_value_cols[-1]

# 숫자 변환
df[col_value] = pd.to_numeric(df[col_value], errors="coerce")

# 3) 시도/시군구 분리 및 정리
##  - '서울특별시 종로구' → 시도='서울특별시', 시군구='종로구'
## - 총계(전국/계 등) 제거

In [ ]:
# 지역 텍스트 정리
def clean_region_name(x: str) -> str:
    s = str(x)
    s = s.replace("\n", " ").replace("\t", " ").strip()
    s = re.sub(r"\s+", " ", s)
    return s

df[col_region] = df[col_region].map(clean_region_name)

# 총합/계/전국 제거
drop_patterns = ["전국", "소계", "합계", "계"]
mask_drop = df[col_region].fillna("").apply(lambda x: any(p in x for p in drop_patterns))
df = df[~mask_drop].copy()

# 시도/시군구 분리: 공백 기준 첫 단어를 시도, 나머지를 시군구로 가정
def split_sido_sigung